In [2]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score

# Aquisição dos dados

In [251]:
test_set = pd.read_csv(r'./eel891-202101-trabalho-1/conjunto_de_teste.csv')
test_set

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,estado_onde_trabalha,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha
0,20001,1,25,presencial,1,M,37,2,0,0,...,,N,,0,0.0,0.0,0.0,0.0,384.0,384.0
1,20002,1,10,internet,1,F,31,2,0,0,...,RJ,N,,0,9.0,5.0,NaN,NaN,275.0,275.0
2,20003,1,10,internet,1,F,18,2,0,0,...,RS,N,,0,9.0,2.0,NaN,NaN,948.0,948.0
3,20004,1,10,presencial,1,F,55,2,0,0,...,,N,,0,9.0,1.0,0.0,0.0,581.0,581.0
4,20005,1,10,presencial,1,F,55,1,0,0,...,,N,,0,0.0,1.0,0.0,0.0,573.0,573.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,24996,1,10,presencial,1,F,49,2,1,0,...,,N,,0,9.0,1.0,0.0,0.0,652.0,652.0
4996,24997,1,5,internet,1,M,18,2,0,0,...,,N,,0,NaN,NaN,NaN,NaN,855.0,855.0
4997,24998,1,10,presencial,1,F,29,1,0,0,...,,N,,0,9.0,5.0,NaN,NaN,371.0,371.0
4998,24999,1,25,internet,1,F,31,2,2,0,...,,N,,0,NaN,NaN,NaN,NaN,670.0,670.0


In [252]:
train_set = pd.read_csv(r'./eel891-202101-trabalho-1/conjunto_de_treinamento.csv')
train_set

,id_solicitante,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,estado_civil,qtde_dependentes,grau_instrucao,...,possui_telefone_trabalho,codigo_area_telefone_trabalho,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
0,1,1,10,presencial,1,M,85,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,600.0,600.0,0
1,2,1,25,internet,1,F,38,1,0,0,...,N,,0,2.0,5.0,NaN,NaN,492.0,492.0,0
2,3,1,20,internet,1,F,37,2,0,0,...,N,,0,NaN,NaN,NaN,NaN,450.0,450.0,1
3,4,1,20,internet,1,M,37,1,1,0,...,Y,54,0,9.0,2.0,NaN,NaN,932.0,932.0,1
4,5,7,1,internet,1,F,51,1,3,0,...,N,,0,9.0,5.0,NaN,NaN,440.0,440.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,19996,1,10,presencial,1,M,27,2,0,0,...,N,,0,9.0,1.0,0.0,0.0,308.0,308.0,0
19996,19997,1,20,presencial,1,F,26,2,1,0,...,N,,0,9.0,2.0,0.0,0.0,639.0,639.0,0
19997,19998,1,10,internet,1,F,63,2,0,0,...,N,,0,9.0,1.0,NaN,NaN,486.0,486.0,0
19998,19999,1,5,internet,1,F,84,1,0,0,...,N,,0,NaN,NaN,NaN,NaN,590.0,590.0,0


In [368]:
training = True

if training:
    df = train_set
else:
    df = test_set

# Análise do dataset

In [369]:
df.describe()

,id_solicitante,produto_solicitado,dia_vencimento,tipo_endereco,idade,estado_civil,qtde_dependentes,grau_instrucao,nacionalidade,tipo_residencia,...,valor_patrimonio_pessoal,possui_carro,meses_no_trabalho,profissao,ocupacao,profissao_companheiro,grau_instrucao_companheiro,local_onde_reside,local_onde_trabalha,inadimplente
count,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.0,20000.000000,19464.000000,...,2.000000e+04,20000.000000,20000.000000,16903.000000,17022.000000,8486.000000,7140.000000,20000.000000,20000.000000,20000.000000
mean,10000.500000,1.284350,13.147250,1.006350,42.352500,2.120850,0.666400,0.0,0.961600,1.261303,...,2.095614e+03,0.339050,0.008900,8.045081,2.533310,3.708107,0.288095,581.295250,581.295250,0.500000
std,5773.647028,1.008239,6.748507,0.079435,14.930177,1.332004,1.236725,0.0,0.202305,0.883580,...,4.403344e+04,0.473399,0.388881,3.210790,1.532765,5.181241,0.944339,227.369798,227.369798,0.500013
min,1.000000,1.000000,1.000000,1.000000,7.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,105.000000,105.000000,0.000000
25%,5000.750000,1.000000,10.000000,1.000000,31.000000,1.000000,0.000000,0.0,1.000000,1.000000,...,0.000000e+00,0.000000,0.000000,9.000000,1.000000,0.000000,0.000000,444.000000,444.000000,0.000000
50%,10000.500000,1.000000,10.000000,1.000000,40.000000,2.000000,0.000000,0.0,1.000000,1.000000,...,0.000000e+00,0.000000,0.000000,9.000000,2.000000,0.000000,0.000000,596.000000,596.000000,0.500000
75%,15000.250000,1.000000,20.000000,1.000000,52.000000,2.000000,1.000000,0.0,1.000000,1.000000,...,0.000000e+00,1.000000,0.000000,9.000000,4.000000,11.000000,0.000000,728.000000,728.000000,1.000000
max,20000.000000,7.000000,25.000000,2.000000,106.000000,7.000000,53.000000,0.0,2.000000,5.000000,...,6.000000e+06,1.000000,32.000000,17.000000,5.000000,17.000000,5.000000,999.000000,999.000000,1.000000


In [370]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,19990,19991,19992,19993,19994,19995,19996,19997,19998,19999
id_solicitante,1,2,3,4,5,6,7,8,9,10,...,19991,19992,19993,19994,19995,19996,19997,19998,19999,20000
produto_solicitado,1,1,1,1,7,1,1,1,2,1,...,1,1,1,1,2,1,1,1,1,2
dia_vencimento,10,25,20,20,1,20,15,5,25,10,...,10,10,5,5,5,10,20,10,5,20
forma_envio_solicitacao,presencial,internet,internet,internet,internet,presencial,presencial,internet,internet,presencial,...,presencial,presencial,internet,internet,presencial,presencial,presencial,internet,internet,presencial
tipo_endereco,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
sexo,M,F,F,M,F,M,F,F,F,M,...,F,M,M,F,M,M,F,F,F,F
idade,85,38,37,37,51,21,64,20,39,44,...,52,48,62,18,23,27,26,63,84,53
estado_civil,2,1,2,1,1,1,4,1,2,2,...,4,2,4,1,2,2,2,2,1,1
qtde_dependentes,0,0,0,1,3,1,2,0,2,2,...,0,2,0,0,0,0,1,0,0,0
grau_instrucao,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [371]:
df.dtypes

id_solicitante                        int64
produto_solicitado                    int64
dia_vencimento                        int64
forma_envio_solicitacao              object
tipo_endereco                         int64
sexo                                 object
idade                                 int64
estado_civil                          int64
qtde_dependentes                      int64
grau_instrucao                        int64
nacionalidade                         int64
estado_onde_nasceu                   object
estado_onde_reside                   object
possui_telefone_residencial          object
codigo_area_telefone_residencial     object
tipo_residencia                     float64
meses_na_residencia                 float64
possui_telefone_celular              object
possui_email                          int64
renda_mensal_regular                float64
renda_extra                         float64
possui_cartao_visa                    int64
possui_cartao_mastercard        

In [372]:
null_df=df.isnull().sum(axis = 0)
null_df[null_df>0]

tipo_residencia                 536
meses_na_residencia            1450
profissao                      3097
ocupacao                       2978
profissao_companheiro         11514
grau_instrucao_companheiro    12860
dtype: int64

# Categorico

In [373]:
df.columns[df.dtypes == object]

Index(['forma_envio_solicitacao', 'sexo', 'estado_onde_nasceu',
       'estado_onde_reside', 'possui_telefone_residencial',
       'codigo_area_telefone_residencial', 'possui_telefone_celular',
       'vinculo_formal_com_empresa', 'estado_onde_trabalha',
       'possui_telefone_trabalho', 'codigo_area_telefone_trabalho'],
      dtype='object')

## Categorico Nominal

In [374]:

colunas_categorias_nominais = [
    'produto_solicitado', 'dia_vencimento', 'forma_envio_solicitacao', 'tipo_endereco', 'sexo', 'estado_civil', 'nacionalidade', 'estado_onde_nasceu',
    'estado_onde_reside', 'possui_telefone_residencial', 'tipo_residencia', 'vinculo_formal_com_empresa',
    'possui_telefone_trabalho','possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes','possui_email',
    'possui_cartao_visa','possui_cartao_mastercard', 'possui_carro', 'possui_telefone_trabalho', 'vinculo_formal_com_empresa',
    'profissao', 'ocupacao', 'local_onde_reside', 'local_onde_trabalha'
]

# Numerico

### Analisando a cardinalidade com intuito de remover alguns campos com alta cardinalidade.

In [375]:

#verificando a cardinalidade de cada variável categórica
r = []
for i in list(df.columns):
    categorias = df[i].unique()
    quantidade = len(categorias)
    r.append({"variavel": i, "card":quantidade})
pd.DataFrame(r)

,variavel,card
0,id_solicitante,20000
1,produto_solicitado,3
2,dia_vencimento,6
3,forma_envio_solicitacao,3
4,tipo_endereco,2
5,sexo,4
6,idade,84
7,estado_civil,8
8,qtde_dependentes,15
9,grau_instrucao,1


## Dessa forma, podemos dropar colunas categoricas numéricas com cardinalidade muito alta
- local_onde_trabalha
- local_onde_reside
- estado_onde_nasceu
## Além disso, foram removidas as colunas por não possuírem relevância ou terem falhas na obtenção dos dados, como de acordo com a tabela de descrição:
- id_solicitante
- profissao_companheiro
- grau_instrucao_companheiro
- possui_telefone_celular
- codigo_area_telefone_trabalho
- codigo_area_telefone_residencial
- grau_instrucao
- estado_onde_trabalha
- meses_no_trabalho
- estado_civil

In [376]:
columns_to_exclude = [
    'id_solicitante',
    'estado_onde_trabalha',
    'profissao_companheiro',
    'grau_instrucao_companheiro',
    'possui_telefone_celular',
    'possui_telefone_residencial',
    'possui_telefone_trabalho',
    'meses_no_trabalho',
    'codigo_area_telefone_trabalho',
    'codigo_area_telefone_residencial',
    'grau_instrucao', 
    'local_onde_trabalha',
    'local_onde_reside',
    'estado_civil','estado_onde_reside','estado_onde_nasceu'
]

df = df.drop(columns=columns_to_exclude, axis=1)

# Binarios

In [377]:

colunas_binarias = [
       'vinculo_formal_com_empresa', 
       'possui_cartao_diners', 'possui_cartao_amex', 'possui_outros_cartoes', 'possui_email',
       'possui_cartao_visa', 'possui_cartao_mastercard', 'possui_carro',
       'tipo_endereco'
]

In [378]:
binarizador = LabelBinarizer()
for v in colunas_binarias:
    df[v] = binarizador.fit_transform(df[v])
df

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,qtde_dependentes,nacionalidade,tipo_residencia,meses_na_residencia,...,possui_cartao_amex,possui_outros_cartoes,qtde_contas_bancarias,qtde_contas_bancarias_especiais,valor_patrimonio_pessoal,possui_carro,vinculo_formal_com_empresa,profissao,ocupacao,inadimplente
0,1,10,presencial,0,M,85,0,1,1.0,12.0,...,0,0,1,1,0.0,1,0,9.0,1.0,0
1,1,25,internet,0,F,38,0,1,1.0,5.0,...,0,0,0,0,0.0,0,0,2.0,5.0,0
2,1,20,internet,0,F,37,0,1,5.0,1.0,...,0,0,0,0,0.0,0,0,NaN,NaN,1
3,1,20,internet,0,M,37,1,1,1.0,1.0,...,0,0,0,0,0.0,0,1,9.0,2.0,1
4,7,1,internet,0,F,51,3,1,0.0,1.0,...,0,0,0,0,0.0,1,1,9.0,5.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,10,presencial,0,M,27,0,1,2.0,0.0,...,0,0,1,1,0.0,1,1,9.0,1.0,0
19996,1,20,presencial,0,F,26,1,1,1.0,3.0,...,0,0,1,1,0.0,1,1,9.0,2.0,0
19997,1,10,internet,0,F,63,0,1,5.0,25.0,...,0,0,0,0,0.0,0,0,9.0,1.0,0
19998,1,5,internet,0,F,84,0,1,1.0,30.0,...,0,0,0,0,0.0,0,0,NaN,NaN,0


# Numerico

In [379]:
from sklearn import impute
if training:
  numerical_categorical_columns = [
    'idade', 'qtde_dependentes', 'meses_na_residencia', 'renda_mensal_regular',
    'renda_extra', 'qtde_contas_bancarias', 'qtde_contas_bancarias_especiais',
    'valor_patrimonio_pessoal'
  ]
  r = [{"col": col, "percentage of null": (df[col].isnull().sum())/df[col].count()} for col in numerical_categorical_columns]
pd.DataFrame(r)


,col,percentage of null
0,idade,0.000000
1,qtde_dependentes,0.000000
2,meses_na_residencia,0.078167
3,renda_mensal_regular,0.000000
4,renda_extra,0.000000
5,qtde_contas_bancarias,0.000000
6,qtde_contas_bancarias_especiais,0.000000
7,valor_patrimonio_pessoal,0.000000


In [380]:
if training:

    imputer_mean = impute.SimpleImputer(missing_values=np.nan, strategy="mean")
    df[['meses_na_residencia']] = imputer_mean.fit_transform(df[['meses_na_residencia']])

## ONE-HOT Enconding

### Vamos aplicar a metodologia do One-Hot Encoding para as colunas abaixo. No entanto, precisamos checar a existência de valores nulos. 

In [381]:
cols = ['sexo', 'tipo_residencia', 'profissao', 'ocupacao', 'forma_envio_solicitacao']
# df.estado_onde_reside.value_counts(), df.sexo.value_counts(), df.tipo_residencia.value_counts(), df.profissao.value_counts(), df.ocupacao.value_counts()

### Notamos a presença de valores diferentes de M e F na coluna sexo. Dessa forma, substituímos tais valores escolhendo probabilisticamente tais características

In [382]:
def replace_values(x, p, a):
    if x != "M" or x != "F":
        return np.random.choice(p=p, a=a, size=1)[0]
    else:
        return x

In [383]:
if training:
    p = df[df.isin(["M", "F"])].sexo.value_counts()/df[df.isin(["M", "F"])].sexo.count()
    a = df[df.isin(["M", "F"])].sexo.value_counts().index
    df["sexo"] = df.sexo.apply(replace_values, args=(p,a,))
    for col in cols: 
        if df[col].dtypes != object:
            df[col] = df[col].fillna(df[col].mean())
        else:
            df[col] = df[col].fillna(df[col].value_counts().index[0])

df

,produto_solicitado,dia_vencimento,forma_envio_solicitacao,tipo_endereco,sexo,idade,qtde_dependentes,nacionalidade,tipo_residencia,meses_na_residencia,...,possui_cartao_amex,possui_outros_cartoes,qtde_contas_bancarias,qtde_contas_bancarias_especiais,valor_patrimonio_pessoal,possui_carro,vinculo_formal_com_empresa,profissao,ocupacao,inadimplente
0,1,10,presencial,0,F,85,0,1,1.0,12.0,...,0,0,1,1,0.0,1,0,9.000000,1.00000,0
1,1,25,internet,0,M,38,0,1,1.0,5.0,...,0,0,0,0,0.0,0,0,2.000000,5.00000,0
2,1,20,internet,0,F,37,0,1,5.0,1.0,...,0,0,0,0,0.0,0,0,8.045081,2.53331,1
3,1,20,internet,0,M,37,1,1,1.0,1.0,...,0,0,0,0,0.0,0,1,9.000000,2.00000,1
4,7,1,internet,0,M,51,3,1,0.0,1.0,...,0,0,0,0,0.0,1,1,9.000000,5.00000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,10,presencial,0,M,27,0,1,2.0,0.0,...,0,0,1,1,0.0,1,1,9.000000,1.00000,0
19996,1,20,presencial,0,F,26,1,1,1.0,3.0,...,0,0,1,1,0.0,1,1,9.000000,2.00000,0
19997,1,10,internet,0,F,63,0,1,5.0,25.0,...,0,0,0,0,0.0,0,0,9.000000,1.00000,0
19998,1,5,internet,0,M,84,0,1,1.0,30.0,...,0,0,0,0,0.0,0,0,8.045081,2.53331,0


In [384]:
from sklearn import preprocessing


if training:
    one_hot_encoder = preprocessing.OneHotEncoder(handle_unknown="ignore")
    one_hot_encoder.fit(df[cols])

one_hot_encoded_columns = one_hot_encoder.transform(df[cols]).toarray()
# one_hot_encoder.get_feature_names()

ohe_df = pd.DataFrame(one_hot_encoded_columns, columns=one_hot_encoder.get_feature_names())

df = pd.concat([df.reset_index(drop=True), ohe_df], axis=1).drop(cols, axis=1)
df

C:\Users\phelipe.francesco\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,produto_solicitado,dia_vencimento,tipo_endereco,idade,qtde_dependentes,nacionalidade,meses_na_residencia,possui_email,renda_mensal_regular,renda_extra,...,x3_0.0,x3_1.0,x3_2.0,x3_2.533309834332041,x3_3.0,x3_4.0,x3_5.0,x4_correio,x4_internet,x4_presencial
0,1,10,0,85,0,1,12.0,0,480.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,25,0,38,0,1,5.0,1,380.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,1,20,0,37,0,1,1.0,1,600.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,20,0,37,1,1,1.0,1,460.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,7,1,0,51,3,1,1.0,1,687.0,600.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1,10,0,27,0,1,0.0,1,423.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19996,1,20,0,26,1,1,3.0,0,350.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
19997,1,10,0,63,0,1,25.0,1,321.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
19998,1,5,0,84,0,1,30.0,1,380.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Categorico Ordinal

In [192]:
[]

[]

In [385]:

if training:
    shuffled_df = df.sample(frac=1, random_state=12345)
    x = shuffled_df.iloc[:, shuffled_df.columns!='inadimplente']
    y = shuffled_df.iloc[:, shuffled_df.columns=='inadimplente']

    num_lines = shuffled_df.shape[0]
    training_size = round(0.8*num_lines)

    x_training = x.values[:training_size,:-1]
    y_training = y.values[:training_size,-1].ravel()

    x_test = x.values[training_size:,:-1]
    y_test = y.values[training_size:,-1].ravel()
else:
    x = df.iloc[:, df.columns!='inadimplente']
    x_test = x.values[:, :-1]

# CatBooster

In [245]:

from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

In [365]:
if training:
    classifier = CatBoostClassifier()
    classifier.fit(x_training, y_training)

    results = []
    for iterations in range(20, 220, 20):
        for depth in range(2, 14, 2):
            print (iterations, depth)
            print ('--------')
            classifier = CatBoostClassifier(depth=depth, iterations=iterations, learning_rate=0.1, verbose=False)
            classifier.fit(x_training, y_training)
            y_test_answer = classifier.predict(x_test)
            results.append((iterations, depth, accuracy_score(y_test, y_test_answer)))
else:
    y_test_answer = classifier.predict(x_test)

In [401]:
best_choose = sorted(results, key=lambda i: i[-1])
print(best_choose[-1])

(80, 2, 0.5675)


In [405]:
if training:
    classifier = CatBoostClassifier(depth=2, iterations=80, learning_rate=0.1, verbose=False)
    classifier.fit(x_training, y_training)
    y_test_answer = classifier.predict(x_test)



In [406]:

if training:
    correct_guesses = sum(y_test_answer == y_test)
    print('Acuracia:', sum(y_test_answer == y_test)/len(y_test))

Acuracia: 0.5635


In [366]:
if not training:
    pd.DataFrame(
    {
        'id_solicitante': np.arange(20001, 25001),
        'inadimplente': y_test_answer,
    }
).to_csv('teste_kaggle_trab1.csv', index=False)

# KNN

In [390]:
acuracia = []

for k in range(1,50,2):
    classificador = KNeighborsClassifier(n_neighbors=k, weights='uniform')
    classificador = classificador.fit(x_training, y_training)
    
    y_resposta_treino = classificador.predict(x_training)
    y_resposta_teste = classificador.predict(x_test)

    print('neighbors', k)

    print(accuracy_score(y_test, y_resposta_teste))
    
    acuracia.append(
        accuracy_score(y_test, y_resposta_teste)
    )


neighbors 1
0.50475
neighbors 3
0.50825
neighbors 5
0.51725
neighbors 7
0.5185
neighbors 9
0.519
neighbors 11
0.5185
neighbors 13
0.5265
neighbors 15
0.5285
neighbors 17
0.53075
neighbors 19
0.526
neighbors 21
0.53225
neighbors 23
0.53125
neighbors 25
0.53375
neighbors 27
0.533
neighbors 29
0.5385
neighbors 31
0.5355
neighbors 33
0.543
neighbors 35
0.53975
neighbors 37
0.54175
neighbors 39
0.544
neighbors 41
0.541
neighbors 43
0.538
neighbors 45
0.53425
neighbors 47
0.541
neighbors 49
0.539


In [400]:
pd.DataFrame(index=pd.RangeIndex(1, 50, 2), data={"acuracia":acuracia}).max()

acuracia    0.544
dtype: float64

# Resultado Final

## De acordo com as análises acima, podemos perceber que o melhor método é o CatBooster, devido a maior acurácia apresentada. 

# Cross Validation

In [407]:
from sklearn.model_selection import cross_val_score

In [408]:
scores = cross_val_score(
    classifier,
    x.values,
    y.values.ravel(),
    cv=8,
)

print (
    'scores =',scores,
    'acurácia média = %6.1f' % (100*sum(scores)/8)
    )

scores = [0.5716 0.5532 0.5776 0.5644 0.5564 0.5616 0.5648 0.5636] acurácia média =   56.4
